In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.elnuevodia.com/topicos/seguridad/'
html = requests.get(url)

In [3]:
html

<Response [200]>

In [4]:
soup = BeautifulSoup(html.text, 'html.parser')

In [5]:
articles = soup.find_all('article', {'class': 'standard-teaser-container condensed-horizontal news'})

In [6]:
for artcl in articles:
      try:
            if artcl.find("h5", {"class": "standard-teaser-subheading slug"}).text == "Seguridad":
                  title= artcl.find('h3', {'class':'standard-teaser-headline teaser-headline'}).text
                  artcl_url= artcl.find("a", href= True)['href']
                  author= artcl.find("div", {"class": "authors-byline-text"}).text
                  print("Titulo: ", title)
                  print("Autor: ", author)
                  print("URL: ", "https://www.elnuevodia.com" + artcl_url)

      except AttributeError:
            print("nada")


Titulo:  Muere mujer de 37 años que impactó con su guagua una barrera de metal en Humacao  
Autor:  Por Yaritza Rivera Clemente
URL:  https://www.elnuevodia.com/noticias/seguridad/notas/muere-mujer-de-37-anos-que-impacto-con-su-guagua-una-barrera-de-metal-en-humacao/
Titulo:  Primer feminicidio del 2024 se convierte en masacre
Autor:  Por Yaritza Rivera Clemente
URL:  https://www.elnuevodia.com/noticias/seguridad/notas/primer-feminicidio-del-2024-se-convierte-en-masacre/
Titulo:  No descartan ningún móvil en asesinato de contratista en Yabucoa: los gatilleros acecharon a la víctima
Autor:  Por Yaritza Rivera Clemente
URL:  https://www.elnuevodia.com/noticias/seguridad/notas/no-descartan-ningun-movil-en-asesinato-de-contratista-en-yabucoa-los-gatilleros-acecharon-a-la-victima/
Titulo:  Golpe a la ganga Los Viraos: Policía le tumba control de acceso ilegal que bloqueaba entrada a residencial en Cayey 
Autor:  Por Yaritza Rivera Clemente
URL:  https://www.elnuevodia.com/noticias/seguridad